In [ ]:
# 1. 
def dfs(current, end, adj_mat, visited):
    visited.add(current)
    if current == end:
        return 0
    for n in adj_mat[current]:
        neighbor = n[0]
        weight = n[1]
        if neighbor not in visited:
            distance = dfs(neighbor, end, adj_mat,visited)
            if distance >= 0:
                return weight + distance
    return -1
        
    
# Insert your DistancesBetweenLeaves function here, along with any subroutines you need
def DistancesBetweenLeaves(n: int, edge_list: List[WeightedEdge]) -> DistMatrix:
    distance = [[0 for _ in range(n)] for _ in range(n)]
    adj_mat = {0:[]}
    for edge in edge_list:
        if edge[0] not in adj_mat.keys():
            adj_mat[edge[0]] = []
        adj_mat[edge[0]].append([edge[1],edge[2]])
        if edge[1] not in adj_mat.keys():
            adj_mat[edge[1]] = []
        adj_mat[edge[1]].append([edge[0],edge[2]])
   
    for n1 in range(0, n):
        for n2 in range(n1+1, n): 
            visited = set()
            dist = dfs(n1, n2, adj_mat, visited)
            distance[n1][n2] = dist
            distance[n2][n1] = dist       
        
    return distance      
#2. 
def LimbLength(j: int, D: DistMatrix) -> float:
    size = len(D)
    leaves = [k for k in range(size) if k != j]
    # limbLength(j) is the min of (Di,j + Dj,k − Di,k)/2
    limbLength = float('inf')
    if len(leaves) == 1:
        limbLength = D[j][leaves[0]]
    for i in range(len(leaves)):
        for k in range(1,len(leaves)):
            i_index = leaves[i]
            k_index = leaves[k]
            length = D[i_index][j]+D[j][k_index]-D[i_index][k_index]
            limbLength = min(0.5*length, limbLength)
    return limbLength


#3. 
def find_min(matrix, indexList):
    min_val = float('inf')
    min_index = [0,0]
    
    for i in range(len(indexList)):
        for j in range(len(indexList)):
            if matrix[i][j] < min_val and i != j:
                min_val = matrix[i][j]
                min_index = [i,j]
    cluster_index = [indexList[min_index[0]], indexList[min_index[1]]]
    return min_val, min_index, cluster_index
        
def modify_matrix(matrix, i, j, new_row, new_col):
    index = sorted([i, j]) # Ensure we remove the larger index first
    # Remove specified row i,j
    matrix.pop(index[1])
    matrix.pop(index[0])
    # Remove specified column i,j from remaining rows
    for row in matrix:
        row.pop(index[1])
        row.pop(index[0])
    # Add new row
    matrix.append(new_row)
    # Add new column
    for row_index, row in enumerate(matrix):
        row.append(new_col[row_index])
    return matrix

# Insert your UPGMA function here, along with any subroutines you need
def UPGMA(D):
    length = len(D) 
    # this one is to record the cluster relationship
    clusters = {i:[i] for i in range(length)}
    T = {}
    # this one is to record the current index of the distance matrix, no need since we use pop, the index will be stored when modifying matrix
    # distanceIndex = [i for i in range(length)]
    age = [0 for _ in range(length)]
    # n is the index of the newly generated internal node
    n = length - 1
    while len(clusters) > 1:
        # find the closest two clusters
        n += 1
        closeset_dist, col_index, cluster_index = find_min(D, list(clusters.keys()))
        # assign new ages to the new clusters
        age_newCluster = closeset_dist/2
        age.append(age_newCluster)
       
        restClusters = [k for k in range(len(D)) if k not in col_index]
        i = col_index[0]
        j = col_index[1]
        # record the internal node edges connecting leaves in T
        T[n] = [cluster_index[0],cluster_index[1]]
        noCi = len(clusters[cluster_index[0]])
        noCj = len(clusters[cluster_index[1]])
        new_row = [(D[i][m] *noCi + D[j][m] *noCj) / (noCi+noCj) for m in restClusters]
        new_col = new_row.copy()
        new_col.append(0)
        # remove the rows and columns of the closest two clusters in the distance matrix 
        # add new row and columns representing the new cluster into the distance matrix
        D = modify_matrix(D, i, j, new_row, new_col)
        # remove the two clusters from the clusters list and store the information into the new cluster
        # add the new cluster into the cluster relationship record
        new_cluster = clusters.pop(cluster_index[0])+ clusters.pop(cluster_index[1])
        clusters[n] = new_cluster
        
    weightedE = []
    for c in T.keys():
        for node in T[c]:
            edge = [c, node]
            weight = age[c]-age[node]
            edge.append(round(weight,2))
            weightedE.append(edge)
    return weightedE